In [286]:
import numpy as np
from sklearn.metrics import pairwise_distances

from phantom.common.distribute import HighsPySolver

In [287]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [288]:
size = 256
keys = [(i, i) for i in range(size)]

In [289]:
lookup_dict = {k: k for k in keys}

In [290]:
from functools import cache


@cache
def lookup_cache(key):
    return lookup_dict.get(key)

In [291]:
lookup_array = np.zeros((size, size), dtype="object")
for k in keys:
    lookup_array[k] = k

In [292]:
%%timeit
lookup_dict.get((100, 100))

29.6 ns ± 0.855 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [293]:
%%timeit
lookup_cache((100, 100))

46.2 ns ± 0.886 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [294]:
%%timeit
lookup_array[100, 100]

36.8 ns ± 0.645 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [295]:
%%timeit
HighsPySolver(60, 60, True)

2025-06-29 15:55:21.379 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True
2025-06-29 15:55:21.454 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True
2025-06-29 15:55:21.536 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True
2025-06-29 15:55:21.609 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True
2025-06-29 15:55:21.684 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True
2025-06-29 15:55:21.758 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True
2025-06-29 15:55:21.834 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True
2025-06-29 15:55:21.914 | D

76.8 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [296]:
problem = HighsPySolver(60, 60, True)

2025-06-29 15:55:27.618 | DEBUG    | phantom.common.distribute:__init__:16 - Compiling highspy problem with n=60, m=60, include_total=True


In [297]:
%%timeit
problem.solve(np.ones((60, 60)), np.ones(60))

2025-06-29 15:55:27.721 | DEBUG    | phantom.common.distribute:solve:77 - Optimal objective = 60.0
2025-06-29 15:55:27.722 | DEBUG    | phantom.common.distribute:solve:78 - Iteration count = 60
2025-06-29 15:55:27.724 | DEBUG    | phantom.common.distribute:solve:77 - Optimal objective = 60.0
2025-06-29 15:55:27.725 | DEBUG    | phantom.common.distribute:solve:78 - Iteration count = 60
2025-06-29 15:55:27.729 | DEBUG    | phantom.common.distribute:solve:77 - Optimal objective = 60.0
2025-06-29 15:55:27.730 | DEBUG    | phantom.common.distribute:solve:78 - Iteration count = 60
2025-06-29 15:55:27.733 | DEBUG    | phantom.common.distribute:solve:77 - Optimal objective = 60.0
2025-06-29 15:55:27.733 | DEBUG    | phantom.common.distribute:solve:78 - Iteration count = 60
2025-06-29 15:55:27.735 | DEBUG    | phantom.common.distribute:solve:77 - Optimal objective = 60.0
2025-06-29 15:55:27.736 | DEBUG    | phantom.common.distribute:solve:78 - Iteration count = 60
2025-06-29 15:55:27.738 | DEBU

3.09 ms ± 150 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [298]:
from collections import defaultdict
from collections.abc import Hashable, Sequence, Set

from scipy.linalg import null_space


def graph_components(adjacency_matrix: np.ndarray) -> Set[Sequence[int]]:
    assert adjacency_matrix.ndim == 2
    assert adjacency_matrix.shape[0] == adjacency_matrix.shape[1]
    adjacency_matrix.shape[0]

    degrees = np.sum(adjacency_matrix, axis=0)
    laplacian = np.diag(degrees) - adjacency_matrix
    kernel_basis = null_space(laplacian, check_finite=False, overwrite_a=True)

    components_dict = defaultdict[Hashable, Sequence[int]](list)
    for i, eigencoords in enumerate(kernel_basis):
        key = tuple(round(x, 8) for x in eigencoords)
        components_dict[key].append(i)

    compositions_set = set(map(tuple, map(sorted, components_dict.values())))
    return compositions_set

In [ ]:
def graph_components_opt(adjacency_matrix: np.ndarray) -> Set[Sequence[int]]:
    components = []
    for i in range(adjacency_matrix.shape[0]):
        connected_to = set(np.nonzero(adjacency_matrix[i, :i])[0])
        new_component = {i}
        for c in components:
            if c & connected_to:
                components.remove(c)
                new_component.update(c)
        components.append(new_component)
    return set(map(tuple, map(sorted, components)))

In [ ]:
def random_graph():
    position_count = 1000
    rng = np.random.default_rng()
    positions = rng.uniform(0, 1000, size=(position_count, 2))
    distances = pairwise_distances(positions)
    adjacency_matrix = np.where(distances < 10, 1, 0)
    return adjacency_matrix

In [301]:
g = random_graph()

In [312]:
graph_components(g)

{(0, 14, 16, 28, 58),
 (1,
  2,
  3,
  4,
  5,
  7,
  17,
  18,
  19,
  22,
  24,
  25,
  26,
  29,
  31,
  36,
  37,
  38,
  42,
  44,
  45,
  46,
  47,
  49,
  50,
  56,
  57,
  59,
  61,
  62,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  83,
  84,
  86,
  87,
  88,
  89,
  90,
  93,
  98),
 (6, 8, 12, 27, 34, 53, 65),
 (9, 30, 35, 60, 96),
 (10,),
 (11, 15, 23, 48, 85, 94, 97),
 (13, 33, 64),
 (20,),
 (21, 51, 54, 82, 91),
 (32,),
 (39, 99),
 (40,),
 (41,),
 (43,),
 (52, 81),
 (55, 63),
 (66, 92),
 (95,)}

In [313]:
graph_components_opt(g)

{(0, 14, 16, 28, 58),
 (1,
  2,
  3,
  4,
  5,
  7,
  17,
  18,
  19,
  22,
  24,
  25,
  26,
  29,
  31,
  36,
  37,
  38,
  42,
  44,
  45,
  46,
  47,
  49,
  50,
  56,
  57,
  59,
  61,
  62,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  83,
  84,
  86,
  87,
  88,
  89,
  90,
  93,
  98),
 (6, 8, 12, 27, 34, 53, 65),
 (9, 30, 35, 60, 96),
 (10,),
 (11, 15, 23, 48, 85, 94, 97),
 (13, 33, 64),
 (20,),
 (21, 51, 54, 82, 91),
 (32,),
 (39, 99),
 (40,),
 (41,),
 (43,),
 (52, 81),
 (55, 63),
 (66, 92),
 (95,)}

In [ ]:
graph_components(g) == graph_components_opt(g)

True

In [ ]:
%%timeit
graph_components_opt(random_graph())

559 μs ± 6.05 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
graph_components(random_graph())

11.5 ms ± 269 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f graph_components graph_components(g)

Timer unit: 1e-07 s

Total time: 0.0210388 s
File: C:\Users\volke\AppData\Local\Temp\ipykernel_5556\1078046343.py
Function: graph_components at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def graph_components(adjacency_matrix: np.ndarray) -> Set[Sequence[int]]:
     7         1         22.0     22.0      0.0      assert adjacency_matrix.ndim == 2
     8         1         14.0     14.0      0.0      assert adjacency_matrix.shape[0] == adjacency_matrix.shape[1]
     9         1          3.0      3.0      0.0      adjacency_matrix.shape[0]
    10                                           
    11         1        636.0    636.0      0.3      degrees = np.sum(adjacency_matrix, axis=0)
    12         1        461.0    461.0      0.2      laplacian = np.diag(degrees) - adjacency_matrix
    13         1     130585.0 130585.0     62.1      kernel_basis = null_space(laplacian, check_finite=False, overwrite_a=True)
    14

In [ ]:
%lprun -f graph_components_opt graph_components_opt(g)

Timer unit: 1e-07 s

Total time: 0.0011628 s
File: C:\Users\volke\AppData\Local\Temp\ipykernel_5556\1708995945.py
Function: graph_components_opt at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def graph_components_opt(adjacency_matrix: np.ndarray) -> Set[Sequence[int]]:
     2         1         25.0     25.0      0.2      n = adjacency_matrix.shape[0]
     3         1          3.0      3.0      0.0      components = []
     4       101        146.0      1.4      1.3      for i in range(n):
     5       100       2908.0     29.1     25.0          connected_to = set(np.nonzero(adjacency_matrix[i, :i])[0])
     6       100        183.0      1.8      1.6          new_component = {i}
     7      2165       3479.0      1.6     29.9          for c in list(components):
     8      2065       4107.0      2.0     35.3              if c & connected_to:
     9        76        260.0      3.4      2.2                  compon